## [ 트랜잭션 제어와 세션]
---
### [ 1. 하나의 단위로 데이터를 처리하는 트랜잭션 ]
#### (1) `트랜잭션(transaction)`이란?
- 더 이상 분할할 수 없는 ☑(최소 수행 단위)를 뜻하며  
계좌 이체와 같이 하나의 작업 또는 밀접하게 연관된 작업을 수행하기 위해  
☑(한 개 이상의 데이터 조작 명령어(DML)로 이루어진다.)


- 트랜잭션은 하나의 트랜잭션 내에 있는 여러 명령어를 한 번에 수행하여 작업을 완료하거나 아예 모두 수행하지 않는 상태, 즉 모든 작업을 취소한다.


- 트랜잭션을 제어하기 위해 사용하는 명령어를 `TCL(Transaction Control Language)`라고 한다.


- 트랜잭션은 SCOTT같은 데이터베이스 계정을 통해 접속하는 동시에 시작된다.  
트랜잭션이 종료되기 전까지 여러 SQL문을 실행하고  
☑(트랜잭션을 제어하는 명령(TCL)을 실행할 때 기존 트랜잭션이 끝난다.)  
그 후 새로운 트랜잭션이 다시 시작한다.

---
### [ 2. 트랜잭션을 제어하는 명령어 ]
- 트랜잭션 제어 명령어 :  
데이터 조작 상태를 '모든 명령어가 정상적으로 수행 완료된 상태'  
또는 '모든 명령어가 수행되지 않아 취소된 상태' 중 하나로 유도하는 명령어이다.


- 수행에 앞서 트랜잭션 제어 명령어를 사용하는 DEPT_TCL 테이블을 만든다.

In [ ]:
--- 11-1
CREATE TABLE DEPT_TCL
    AS SELECT * FROM DEPT;

SELECT * FROM DEPT_TCL;

결과 : DEPT 테이블이 복사됨

![2(1)](https://user-images.githubusercontent.com/53929665/94340959-d36d6180-0040-11eb-8009-8e1c8086171c.JPG)

- DEPT_TCL 테이블에 데이터를 입력/수정/삭제 하기

In [ ]:
--- 11-2
INSERT INTO DEPT_TCL VALUES (50, 'DATABASE', 'SEOUL');

UPDATE DEPT_TCL SET LOC = 'BUSAN' WHERE DEPTNO=40;

DELETE FROM DEPT_TCL WHERE DNAME = 'RESEARCH';

SELECT * FROM DEPT_TCL;

결과 :  
50번 부서가 새로 추가되었고 40번 부서의 LOC열이 BUSAN으로 수정되었으며 20번 부서는 삭제되었다.

![2(2)](https://user-images.githubusercontent.com/53929665/94340961-d49e8e80-0040-11eb-893b-9f13e0bccd17.JPG)

#### (1) 트랜잭션을 취소하고 싶을 때는 ROLLBACK
- ☑(위의 수행에서 실행한 세 개의 데이터 조작어는 중간에 별다른 작업이 없었다면 하나의 트랜잭션에 속해 있을 것이다.)
- 만약, 이 모든 작업의 수행을 취소하고 싶다면 `ROLLBACK`명령어를 사용한다.
    - `ROLLBACK`은 현재 트랜잭션에 포함된 데이터 조작 관련 명령어의 수행을 모두 취소한다.

In [ ]:
--- 11-3
ROLLBACK;

SELECT * FROM DEPT_TCL;

결과 : 앞에서 실행한 데이터 조작 관련 명령어 실행이 모두 취소되었다.

![2-1](https://user-images.githubusercontent.com/53929665/94340962-d49e8e80-0040-11eb-8a4c-47997f766b8e.JPG)

#### (2) 트랜잭션을 영원히 반영하고 싶을 때는 COMMIT
- `COMMIT`명령어 : `ROLLBACK`과 달리 지금까지 수행한 트랜잭션 명령어를 데이터베이스에 영구히 반영하고 싶을 때 사용한다.

<br>

- `COMMIT` 수행에 앞서 다시 DEPT_TCL 테이블에 데이터 입력/수정/삭제하기

In [ ]:
--- 11-4
INSERT INTO DEPT_TCL VALUES (50, 'NETWORK', 'SEOUL');

UPDATE DEPT_TCL SET LOC = 'BUSAN' WHERE DEPTNO=20;

DELETE FROM DEPT_TCL WHERE DEPTNO = 40;

SELECT * FROM DEPT_TCL;

결과 :

![2-2(1)](https://user-images.githubusercontent.com/53929665/94340964-d5372500-0040-11eb-992b-4699169fa349.JPG)

- `COMMIT`으로 명령어 반영하기

In [ ]:
--- 11-5
COMMIT;

결과 :
- 이 시점부터 앞서 실행한 명령어는 취소가 불가능하다.
- 그러므로 `COMMIT`은 트랜잭션 작업이 정상적으로 수행되었다고 확신할 때 사용해야 한다.
- `COMMIT`으로 실행한 내용의 반영은 되돌릴 수 없으므로 정말 신중하게 진행해야한다.

![2-2(2)](https://user-images.githubusercontent.com/53929665/94340965-d5372500-0040-11eb-80ff-9360e2a26aa4.JPG)


---
### [ 3. 세션과 읽기 일관성의 의미 ]

#### (1) 세션이란?
- 세션(session)

    - 일반적 의미 : 어떤 활동을 위한 시간이나 기간
    - 오라클에서의 의미 : 데이터베이스 접속을 시작 ~ 접속을 종료하기 까지의 전체 수행 시간을 의미
        
        
- 따라서,  
세션이 여러 개 = 현재 오라클 데이터 베이스에 접속하여 사용 중인 연결이 여러 개 있다  
ex) 토드를 사용하여 SCOTT계정 사용, SQL PLUS에서 SCOTT으로 접속 = 세션 2개


- 또한,  
`트랜잭션`은 데이터 조작 명령어가 모인 하나의 작업 단위를 뜻하며  
`세션` 내부에는 ☑(하나 이상의 트랜잭션이 존재한다.)

#### (2) 읽기 일관성의 중요성
- 데이터베이스는 보통 여러 곳(여러 사용자, 여러 응용 프로그램)에서 동시에 접근하여 데이터를 관리/사용하는 것이 목적이므로 대부분 수많은 세션이 동시에 연결되어 있다. 


- `읽기 일관성(Read Consistency)` :  
어떤 특정 세션에서 테이블의 데이터를 변경 중일 때  
그 외 다른 세션에서는 데이터의 변경이 확정되기 전까지 변경 사항을 알 필요가 없으므로,  
데이터를 변경 중인 세션을 제외한 나머지 세션에서는  
☑(현재 진행 중인 변경과 무관한 본래의 데이터를 보여 주는 특성을 의미한다.)


- 수행에 앞서,  
토드를 사용한 세션을 세션A / SQLPLUS를 사용한 세션을 세션B라 지정한다.


![3(1)](https://user-images.githubusercontent.com/53929665/94341484-ab7ffd00-0044-11eb-948c-a4ed5901f9f9.JPG)

- 세션 A에서 `DELETE`문을 사용하여 50번 부서 데이터를 삭제한 후에  
각 세션에서 DEPT_TCL 테이블을 조회했을 경우 결과는 다음과 같다.

결과 :
- 세션 A에서는 50번 부서가 삭제된 상태로 DEPT_TCL테이블이 조회되지만
- 세션 B에서는 50번 부서가 아직 삭제되지 않은 상태, 즉 변경이 일어나기 전 상태로 출력된다.


![3(2)](https://user-images.githubusercontent.com/53929665/94341486-acb12a00-0044-11eb-9bba-4e62fe348491.JPG)

- 이 결과는 세션 A에서 실행한 `DELETE`문의 결과가 데이터베이스에 완벽하게 반영되지 않았기 때문이다.  
즉, `COMMIT`되지 않았다는 이야기이다.  


- 실행한 `DELETE`문의 실행 결과가 데이터베이스에 영구히 반영되기 전까지  
`DELETE`를 실행한 세션A를 제외한 다른 세션에서는 50번 부서 데이터의 변화를 확인할 수 없다.  
따라서, 50번 부서의 삭제를 확인할 수 있는 것은 세션A뿐이다.


- 마지막으로, `COMMIT`했을 때 다음과 같이  
세션 B에서도 50번 부서가 삭제된 채 조회되는 것을 확인할 수 있다.



![3(3)](https://user-images.githubusercontent.com/53929665/94341488-acb12a00-0044-11eb-85e6-e28c4cc928c5.JPG)


- 데이터를 직접 변경 중인 해당 세션을 제외한 모든 세션은 다른 세션의 데이터 변경과 상관없이 이미 확정된 데이터만 검색됨으로써 `읽기 일관성`을 보장할 수 있다.

---
### [ 4. 수정 중인 데이터 접근을 막는 LOCK ]
#### (1) LOCK이란?
- `LOCK` :  
특정 세션에서 조작 중인 데이터는 트랜잭션이 완료(COMMIT, ROLLBACK)되기 전까지  
다른 세션에서 조작할 수 없는 상태  
즉, ☑(조작 중인 데이터를 다른 세션은 조작할 수 없도록 접근을 보류시키는 것)


#### (2) LOCK 개념
- 세션 A에서 `UPDATE`문을 사용하여 데이터를 변경

결과 :
- 앞의 세션 때와 마찬가지로 `UPDATE`문을 실행하고 있는 세션 A에서는 30번 부서의 LOC가 SEOUL로 변경되었지만  
`COMMIT`되지 않은 상태이므로 세션 B에서는 30번 부서에 변화가 없다.

![4-2(1)](https://user-images.githubusercontent.com/53929665/94341850-c2741e80-0047-11eb-95f4-de37db092225.JPG)

- 위의 상태에서 세션 B의 30번 부서(세션 A가 변경 중인)에 `UPDATE`문을 실행하면 다음과 같다.

결과 :
- 실행하면 다음과 같이 아무런 동작이 일어나지 않는다.

![4-2(2)](https://user-images.githubusercontent.com/53929665/94341852-c30cb500-0047-11eb-9aa1-836228d3cfbd.JPG)

- 왜냐하면, 세션 A에서 DEPT_TCL테이블의 30번 부서 데이터를 먼저 조작하고 있기 때문이다.


- 세션 A에서 `COMMIT`/`ROLLBACK`을 수행하기 전까지  
30번 부서 행 데이터를 조작하려는 다른 세션은 위의 화면과 같이 작업을 대기하게 된다.  
이렇게 특정 세션에서 데이터 조작이 완료될 때까지 다른 세션에서 해당 데이터 조작을 기다리는 현상을  
`HANG(행)`이라고 지칭한다.


- 따라서, 세션 A에서 `COMMIT`으로 데이터 변경을 확정하거나, `ROLLBACK`으로 실행을 취소해야만  
30번 부서 데이터의 `LOCK`이 풀린다.  
그리고, 데이터의 `LOCK`이 풀린 즉시 세션 B는 UPDATE 문을 실행한다.


- 세션 A에서 `COMMIT`명령어를 실행해보면 세션 B에서는 다음과 같은 일이 일어난다.

결과 :
- 세션 A에서 `COMMI`명령어를 실행하는 순간  
세션 B의 `UPDATE`문이 실행된 것을 확인할 수 있다.
![4-2(3)](https://user-images.githubusercontent.com/53929665/94341943-9a38ef80-0048-11eb-8488-e73a2f985f69.JPG)

- 마지막으로, 각 세션에서 다시 DEPT_TCL테이블을 조회해보면 다음과 같다.

결과 :
- 세션 A의 `LOCK`이 풀린 후 세션 B의 `UPDATE`문이 실행되었지만  
아직 `COMMIT`을 하지 않은 상태이기 때문에  
세션 B에서만 30번 부서의 DNAME 열이 DATABASE로 출력된다.
![4-2(4)](https://user-images.githubusercontent.com/53929665/94341944-9b6a1c80-0048-11eb-8bb4-0d3650ce3850.JPG)

- 따라서, 세션 B에서 `COMMIT`을 실행하면 세션 A에서도 다음과 같이 30번 부서의 DNAME이 변경된 것을 확인 할 수 있다.

![4-2(5)](https://user-images.githubusercontent.com/53929665/94341999-fd2a8680-0048-11eb-8180-7b6da9d690a5.JPG)


#### (3) LOCK 종류
- `행 레벨 록(row level lock)` : 앞에서와 같이 특정 행 데이터만 `LOCK`이 발생하는 것


- `테이블 레벨 록(table level lock)` :  
위의 행 레벨 록과 달리 `WHERE`절을 지정하지 않고 데이터 테이블을 조작할 경우,  
테이블에 저장되어 있는 전체 행이 `LOCK`상태가 된다.
    - 하지만, 테이블 전체 행이 `LOCK`상태여도 `INSERT`문은 수행 가능하다.


- ☑ 따라서, 자신이 실행해야 하는 데이터 조작 관련 SQL문을 통해 LOCK이 적용되는 데이터의 범위가 크거나,  
오랜 시간 동안 작업이 진행되어야 한다면 반드시 함께 일하는 개발자들에게 작업 중임을  
미리 알리고 양해를 구하는 센스를 발휘해야한다.